In [ ]:
import requests
import csv
import os

# Define the base URL for the API
base_url = "http://environment.data.gov.uk/hydrology"

# List of GUIDs and their associated keywords
WISKI_keyword_pairs = [
    {"WISKI": "SE62CLO1SS", "keyword": "Carlton Lake Bottom"},

    # Add more WISKI and keyword pairs as needed
]

# Create or open the master CSV file for writing
master_output_file = 'combined_groundwaterlevels_dipped.csv'

# Open the master CSV file for writing
with open(master_output_file, 'w', newline='') as master_file:
    master_writer = csv.writer(master_file)

    # Write the header once at the top of the master file
    master_writer.writerow(["keyword", "date", "value"])

    # Loop through each GUID and keyword pair
    for pair in WISKI_keyword_pairs:
        WISKI = pair["WISKI"]
        keyword = pair["keyword"]

        # Construct the API endpoint URL for historical readings
        endpoint = f"/id/measures/-gw-dipped-i-mAOD-qualified/readings?mineq-date=2023-01-01&max-date=2023-12-31&station.wiskiID={WISKI}"

        # Construct the full URL
        url = f"{base_url}{endpoint}"

        # Send the GET request to the API
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()

            # Define the output CSV file name using the keyword
            output_file = f'{keyword}_groundwaterlevels.csv'

            # Open an individual CSV file for writing
            with open(output_file, 'w', newline='') as file:
                writer = csv.writer(file)

                # Write the header
                writer.writerow(["date", "value"])

                # Write the data
                for reading in data['items']:
                    # Check if both 'dateTime' and 'value' keys exist
                    if 'dateTime' in reading and 'value' in reading:
                        date = reading['dateTime']
                        value = reading['value']
                        writer.writerow([date, value])

                        # Write the same data to the master CSV file with the keyword
                        master_writer.writerow([keyword, date, value])

            print(f"CSV file '{output_file}' has been downloaded and formatted successfully.")
        else:
            print(f"Failed to retrieve data for WISKI {WISKI} with keyword {keyword}: {response.status_code}")

print(f"All data has been combined into '{master_output_file}' successfully.")